In [ ]:
# Authentication Code with ALL Scopes
# IMPORTANT: Delete token.json and enable the Google Sheets API before running!

import os.path
from google.auth.transport.requests import Request
from google.oauth2.credentials import Credentials
from google_auth_oauthlib.flow import InstalledAppFlow
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError

# Add the final scope for reading Google Sheets
SCOPES = [
    'https://www.googleapis.com/auth/classroom.courses.readonly',
    'https://www.googleapis.com/auth/classroom.rosters.readonly',
    'https://www.googleapis.com/auth/classroom.profile.emails',
    'https://www.googleapis.com/auth/classroom.student-submissions.students.readonly',
    'https://www.googleapis.com/auth/spreadsheets.readonly'
]

def authenticate():
    """Authenticates with all required scopes and returns a service object."""
    credentials = None
    if os.path.exists('token.json'):
        # The file token.json stores the user's access and refresh tokens.
        credentials = Credentials.from_authorized_user_file('token.json', SCOPES)
    
    # If there are no (valid) credentials available, let the user log in.
    if not credentials or not credentials.valid:
        if credentials and credentials.expired and credentials.refresh_token:
            credentials.refresh(Request())
        else:
            flow = InstalledAppFlow.from_client_secrets_file(
                'credentials.json', SCOPES)
            credentials = flow.run_local_server(port=0)
        # Save the credentials for the next run
        with open('token.json', 'w') as token:
            token.write(credentials.to_json())
    return credentials

print("Starting authentication for Classroom and Sheets APIs...")
try:
    creds = authenticate()
    print("✓ Authentication successful!")
    
    # Build both service objects now
    classroom_service = build('classroom', 'v1', credentials=creds)
    sheets_service = build('sheets', 'v4', credentials=creds)
    
    print("✓ Classroom and Sheets service objects are ready.")

except FileNotFoundError:
    print("\n[ERROR] `credentials.json` not found.")
except Exception as e:
    print(f"\n[ERROR] An error occurred: {e}")

In [ ]:
# Step 2: Fetch a List of All Courses
# This cell uses the authenticated classroom_service object to retrieve a list of courses
# that the authenticated user is a member of.
def fetch_courses(service_object):
    """Fetches and prints the user's courses."""
    print("\nFetching courses...")
    try:
        courses = []
        page_token = None
        while True:
            response = service_object.courses().list(pageToken=page_token).execute()
            courses.extend(response.get('courses', []))
            page_token = response.get('nextPageToken', None)
            if not page_token:
                break
        
        if not courses:
            print('No courses found.')
            return None
        
        print(f"Found {len(courses)} courses.")
        return courses
        
    except HttpError as error:
        print(f'An error occurred while fetching courses: {error}')
        return None
# Execute the function to fetch courses
all_courses = fetch_courses(classroom_service)

In [ ]:
# This script assumes you have already run the authentication code and have a 
# valid 'classroom_service' object. It also requires the pandas library.

import pandas as pd
from googleapiclient.errors import HttpError
import re

# ------------------------------------------------------------------------------------
# Step 3: Fetch Base Student Data
# First, we get all students from all courses to create a master list.
# We will store it in a dictionary for easy lookup later.
# ------------------------------------------------------------------------------------
def get_all_students(service_object, course_list):
    """Fetches all students from a list of courses and returns them in a dictionary."""
    if not course_list:
        return {}

    print("Fetching base student data from all courses...")
    all_students_dict = {}
    for course in course_list:
        course_id = course.get('id')
        course_name = course.get('name', 'Unnamed Course')
        group = course.get('section', 'Group 1')

        # quick fix remove course group from course anme
        if '-' in course_name:
            course_name = course_name.split('-')[0]
            
        try:
            students = []
            page_token = None
            while True:
                response = service_object.courses().students().list(
                    courseId=course_id, pageToken=page_token).execute()
                students.extend(response.get('students', []))
                page_token = response.get('nextPageToken', None)
                if not page_token:
                    break
            
            for student in students:
                student_id = student.get('userId')
                profile = student.get('profile', {})
                name_dict = profile.get('name', {})
                
                # We only add the student once, but list all courses they are in.
                if student_id not in all_students_dict:
                    all_students_dict[student_id] = {
                        'student_id': student_id,
                        'student_alias': name_dict.get('fullName', 'N/A'),
                        'student_email': profile.get('emailAddress', 'N/A'),
                        'courses': [(course_name, group)]
                    }
                else:
                    all_students_dict[student_id]['courses'].append((course_name, group))

        except HttpError as error:
            print(f'An error occurred fetching students for course {course_name}: {error}')

    print(f"Found {len(all_students_dict)} unique students across all courses.")
    return all_students_dict


# Fetching from Google Sheets and building the DataFrame
# This function will now use the sheets_service
def get_quiz_responses_from_sheet(sheets_service, response_url):
    """
    Dynamically finds all sheets named 'Form Responses X', reads data from each,
    and combines them into a single DataFrame.
    """
    # Step 1: Extract the spreadsheet ID from the URL
    match = re.search(r'/d/([a-zA-Z0-9-_]+)', response_url)
    if not match:
        print("  - Could not parse Spreadsheet ID from URL.")
        return None
    spreadsheet_id = match.group(1)
    try:
        # Step 2: Get spreadsheet metadata to find all sheet names
        sheet_metadata = sheets_service.spreadsheets().get(spreadsheetId=spreadsheet_id).execute()
        sheets = sheet_metadata.get('sheets', '')
        
        # Filter for sheets that match the "Form Responses" pattern
        relevant_sheet_names = [
            s['properties']['title'] for s in sheets 
            if 'Form Responses' in s['properties']['title']
        ]
        if not relevant_sheet_names:
            print(f"  - No sheets found matching the 'Form Responses' pattern.")
            return None
        
        print(f"  + Found {len(relevant_sheet_names)} response sheet(s): {', '.join(relevant_sheet_names)}")
        # Step 3: Read data from each relevant sheet and combine it
        all_rows = []
        header = None
        for sheet_name in relevant_sheet_names:
            # The range A:Z will get all columns
            sheet_range = f"'{sheet_name}'!A:Z"
            result = sheets_service.spreadsheets().values().get(
                spreadsheetId=spreadsheet_id, range=sheet_range).execute()
            
            values = result.get('values', [])
            
            if not values:
                continue # Skip empty sheets
            # Use the header from the first sheet we process
            if header is None:
                header = values[0]
                all_rows.extend(values[1:])
            else:
                # For subsequent sheets, just add the data rows (skip the header)
                all_rows.extend(values[1:])
        
        if not all_rows:
            print("  - All response sheets were empty.")
            return None
        # Step 4: Create the final DataFrame
        sheet_df = pd.DataFrame(all_rows, columns=header)
        return sheet_df
    except HttpError as error:
        print(f"  - An error occurred reading the spreadsheet: {error}")
        return None

def build_final_dataframe(classroom_service, sheets_service, course_list, base_student_df, quiz_title):
    """Finds the quiz, gets the response sheet, and merges data."""
    print(f"\nSearching for quiz titled: '{quiz_title}'...")
    
    full_quiz_df = pd.DataFrame()
    for course in course_list:
        course_id = course.get('id')
        course_name = course.get('name', 'Unnamed Course')
        group = course.get('section', 'Group 1')
        print(f"\nChecking in course: '{course_name}'...")
        try:
            coursework_response = classroom_service.courses().courseWork().list(courseId=course_id).execute()
            quiz = next((cw for cw in coursework_response.get('courseWork', []) if cw.get('title') == quiz_title), None)
            if not quiz:
                print(f"  - Quiz '{quiz_title}' not found in this course.")
                continue
            print(f"  + Found quiz. Checking for response sheet...")
            materials = quiz.get('materials', [])
            response_url = next((mat['form']['responseUrl'] for mat in materials if 'form' in mat and 'responseUrl' in mat['form']), None)
            
            if not response_url:
                print("  - No Google Sheet response URL found for this quiz.")
                continue
            print(f"  + Found response sheet URL. Reading data...")
            # This now calls the new, more powerful function
            quiz_df = get_quiz_responses_from_sheet(sheets_service, response_url)
            
            if quiz_df is not None and not quiz_df.empty:
                full_quiz_df = pd.concat([full_quiz_df, quiz_df], ignore_index=True)
        except HttpError as error:
            print(f"  - An error occurred in course {course_name}: {error}")
    if full_quiz_df.empty:
        print("\nCould not retrieve any quiz data. Returning base student list.")
        return base_student_df
    
    print("\nMerging quiz data with student roster...")
    
    full_quiz_df = full_quiz_df.rename(columns={
        "Email Address": "student_email",
        "Matriculation Number": "matriculation_number",
        "First Name": "first_name",
        "Last Name": "last_name"
    }).drop_duplicates(subset=['student_email'], keep='last') # Keep only the latest submission per student
    final_df = pd.merge(base_student_df, full_quiz_df[['student_email', 'matriculation_number', 'first_name', 'last_name']], on='student_email', how='left')
    return final_df

# --- EXECUTE THE FINAL WORKFLOW ---

# 1. Create a base DataFrame from your existing 'all_students_dict'
all_students_dict = get_all_students(classroom_service, all_courses) # from previous code
base_df = pd.DataFrame(all_students_dict.values())
# base_df['course_name'] = base_df['course_name'].apply(lambda x: ', '.join(x)) # Make courses a string

# 2. Define your quiz title
quiz_to_find = "Registration"

# 3. Build the final, complete DataFrame
complete_df = build_final_dataframe(classroom_service, sheets_service, all_courses, base_df, quiz_to_find)


# 4. Display the result
print("\n\n--- Complete Student DataFrame ---")
# Reorder columns for final display
final_columns = ['student_alias', 'student_email', 'matriculation_number', 'first_name', 'last_name', 'courses']
for col in final_columns:
    if col not in complete_df.columns:
        complete_df[col] = None # Add any missing columns

print(complete_df[final_columns].to_string())


In [ ]:
# Example to just showing students of Group 4
complete_df[complete_df['courses'].astype(str).str.contains('Group 4')].sort_values('last_name')

In [ ]:
print('Missing Registrations:')
print()

for x in list(complete_df[complete_df['courses'].astype(str).str.contains('Digital Electronics')][complete_df.isna().any(axis=1)]['student_email']):
    print(x)